In [1]:
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config
from ray.air.integrations.wandb import WandbLoggerCallback
from ray import tune, train
from envs.gym_env import MyEnv, MyEnv2
from ray.rllib.utils.replay_buffers.replay_buffer import StorageUnit

def env_creator(env_config):
    return MyEnv()

def env_creator2(env_config):
    return MyEnv2()

tune.register_env("my_env", env_creator)
tune.register_env("my_env2", env_creator2)

# Run with:
# python run_regression_tests.py --dir [this file]

num_gpus = 0


# {(pixel-filta+2*padding)/stride}+1
# 64x64x3の画像を入力とする場合の設定
# model_dict = {"conv_filters": [
#             [16, [8, 8], 4],
#             [32, [4, 4], 4],
#             [64, [4, 4], 2],
#             ],}

# 128x128x3の画像を入力とする場合の設定
# model_dict = {"conv_filters": [
#             [16, [8, 8], 4],
#             [32, [4, 4], 3],
#             [64, [3, 3], 2],
#             [256, [6, 6], 1],
#             ],}

config = (
    DreamerV3Config()
    .training(
        # model=model_dict,
        model_size="M", # model_size="XS", # XS, S, M, L, XL
        training_ratio=1024,
        batch_size_B=16,
    )
    .environment(
        env="my_env2",
    )
    .resources(
        num_learner_workers=0 if num_gpus == 1 else num_gpus,
        num_gpus_per_learner_worker=1 if num_gpus else 0,
        num_cpus_for_local_worker=1,
    )
    .rollouts(
        # If we use >1 GPU and increase the batch size accordingly, we should also
        # increase the number of envs per worker.
        num_envs_per_worker=1,
        remote_worker_envs=True,
    )
)

tuner = tune.Tuner(
    "DreamerV3",  # 2. Specify the algorithm name
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 40
            #   "timesteps_total": 10,
              },  # 学習の終了条件
        callbacks=[WandbLoggerCallback(project="Wandb_nearme")]  # コメントアウトを外すとwandbにログが送信される(https://docs.wandb.ai/quickstart)
    ),
    param_space=config
)

results = tuner.fit()

2024-02-26 02:33:25,298	INFO wandb.py:307 -- Already logged into W&B.
(pid=43848) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
(pid=43848)   logger.warn(
(pid=43848) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
(pid=43848)   logger.warn(f"{pre} is not within the observation space.")
(DreamerV3 pid=43848) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\vector\__init__.py:53: UserWarning: WARN: `gymnasium.vector.make(...)` is deprecated and will be replaced by `gymnasium.make_vec(...)` in v1.0
(DreamerV3 pid=43848)   gym.logger.warn(
(DreamerV3 pid=43848) Trainable.setup took 16.190 seconds. If your trainable is slow to initialize, consider 

(DreamerV3 pid=43848) Model: "dreamer_model"
(DreamerV3 pid=43848) _________________________________________________________________
(DreamerV3 pid=43848)  Layer (type)                Output Shape              Param #   
(DreamerV3 pid=43848) =================================================================
(DreamerV3 pid=43848)  world_model (WorldModel)    multiple                  0 (unused)
(DreamerV3 pid=43848) |¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
(DreamerV3 pid=43848) | image_encoder (CNNAtari)  multiple                  1552032   |
(DreamerV3 pid=43848) ||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯||
(DreamerV3 pid=43848) || conv2d (Conv2D)         multiple                  2304      ||
(DreamerV3 pid=43848) ||                                                             ||
(DreamerV3 pid=43848) || conv2d_1 (Conv2D)       multiple                  73728     ||
(DreamerV3 pid=43848) ||                                                   

(_WandbLoggingActor pid=49736) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
(_WandbLoggingActor pid=49736)   logger.warn(
(_WandbLoggingActor pid=49736) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
(_WandbLoggingActor pid=49736)   logger.warn(f"{pre} is not within the observation space.")


(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  3  action:  [-0.7349467  0.3305098]  reward:  0.8756456475999357


(_WandbLoggingActor pid=49736) wandb: Currently logged in as: hirekatsu0523. Use `wandb login --relogin` to force relogin


...

(_WandbLoggingActor pid=49736) wandb: Tracking run with wandb version 0.16.3
(_WandbLoggingActor pid=49736) wandb: Run data is saved locally in C:\Users\alaleh\ray_results\DreamerV3_2024-02-26_02-33-22\DreamerV3_my_env2_facad_00000_0_2024-02-26_02-33-25\wandb\run-20240226_023356-facad_00000
(_WandbLoggingActor pid=49736) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=49736) wandb: Syncing run DreamerV3_my_env2_facad_00000
(_WandbLoggingActor pid=49736) wandb:  View project at https://wandb.ai/hirekatsu0523/Wandb_nearme
(_WandbLoggingActor pid=49736) wandb:  View run at https://wandb.ai/hirekatsu0523/Wandb_nearme/runs/facad_00000


(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  8  action:  [-0.49199873 -0.43193257]  reward:  0.09308249486704556
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  9  action:  [-0.06787804  0.10120105]  reward:  0.3164032465858315
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  10  action:  [0.1579515 0.7846749]  reward:  0.2022062880201801
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  11  action:  [ 0.66945314 -0.5494766 ]  reward:  0.4111605724669818
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  12  action:  [-0.35849217 -0.4959776 ]  reward:  0.4111605724669818
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  13  action:  [-0.55505157  0.00389999]  reward:  0.31343221973329954
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  14  action:  [-0.65369475 -0.7521482 ]  reward:  0.3409687507132241
(Dr

2024-02-26 03:06:13,662	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 4.836 s, which may be a performance bottleneck.
2024-02-26 03:06:13,663	WARNING util.py:202 -- The `process_trial_result` operation took 4.837 s, which may be a performance bottleneck.
2024-02-26 03:06:13,663	WARNING util.py:202 -- Processing trial results took 4.837 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-02-26 03:06:13,664	WARNING util.py:202 -- The `process_trial_result` operation took 4.839 s, which may be a performance bottleneck.
(_QueueActor pid=11476) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
(_QueueActor pid=11476)   logger.warn(
(_QueueActor pid=11476) c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\utils\passive_e

(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  5  action:  [-0.790257    0.31842145]  reward:  0.8063392584903089
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  6  action:  [-1.6406544   0.08626038]  reward:  0.8177223865723977
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  7  action:  [-0.13055807 -0.10633618]  reward:  0.8216307341694356
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  8  action:  [0.53729415 0.29393384]  reward:  0.7461908602187022
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  9  action:  [-0.8298193 -1.1709232]  reward:  0.6938628726425371
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  10  action:  [-0.90115064 -0.56745803]  reward:  0.5242413339804893
(DreamerV3 pid=43848) --------------------
(DreamerV3 pid=43848) episode:  11  action:  [-0.61887187  0.32094225]  reward:  0.3436931227232588
(Dreamer

2024-02-26 10:26:32,470	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
